# Prototipo simulador parte 4: control respiratorio

In [1]:
# Librerías
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Estilo
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
class Paciente:
    """Clase que define un paciente con parámetros normales;
    para agregar pacientes con patologías, crear una subclase con modificaciones
    en los parámetros específicos que haga falta"""
    def __init__(self, R1=5, C1=0.05, R2=10, C2=0.05):
        self.R1 = R1
        self.C1 = C1
        self.R2 = R2
        self.C2 = C2
        self.E1 = 1 / self.C1
        self.E2 = 1 / self.C2

In [3]:
class Ventilador:
    """Parámetros y perfiles de ventilación mecánica."""
    def __init__(self,
                 modo: str = 'PCV',
                 PEEP: float = 5.0,
                 P_driving: float = 15.0,
                 fr: float = 20.0,
                 Ti: float = 1.0,
                 Vt: float = None):
        self.modo = modo
        self.PEEP = PEEP
        self.P_driving = P_driving
        self.fr = fr
        self.Ti = Ti
        self.T_total = 60.0 / fr
        self.Vt = Vt
        if modo == 'VCV':
            assert Vt is not None, "Se requiere Vt para modo VCV"
            self.flow_insp = Vt / Ti
        else:
            self.flow_insp = None

    def presion(self, t: float) -> np.ndarray:
        """Perfil de presión en la vía aérea según el modo y el tiempo t."""
        t_arr = np.asarray(t)
        en_insp = (t_arr % self.T_total) < self.Ti
        if self.modo == 'PCV':
            P_control = self.PEEP + self.P_driving
            return np.where(en_insp, P_control, self.PEEP)
        elif self.modo == 'VCV':
            return np.full_like(t_arr, self.PEEP)
        else:
            raise ValueError(f"Modo desconocido: {modo}")

    def flujo(self, t: float) -> np.ndarray:
        """Perfil de flujo inspirado en VCV, 0 fuera de inspiración."""
        if self.modo != 'VCV':
            return np.zeros_like(np.asarray(t))
        t_arr = np.asarray(t)
        return np.where((t_arr % self.T_total) < self.Ti, self.flow_insp, 0.0)

In [4]:
class Simulador:
    """Orquesta la simulación paciente-ventilador."""
    def __init__(self,
                 paciente: Paciente,
                 ventilador: Ventilador,
                 control: 'ControlRespiratorio' = None):
        self.paciente = paciente
        self.ventilador = ventilador
        self.control = control
        if ventilador.modo == 'ESP':
            assert self.control is not None, "Se requiere un módulo de ControlRespiratorio para el modo 'ESP'"

    def _modelo_edo(self, t, y):
        V1, V2 = y

        if self.ventilador.modo == 'ESP':
            # En modo espontáneo, la presión la genera el módulo de control
            P_aw = self.control.generar_Pmus(t)
        elif self.ventilador.modo == 'VCV':
            # Lógica para VCV
            en_insp = (t % self.ventilador.T_total) < self.ventilador.Ti
            flow_total = np.where(en_insp, self.ventilador.flow_insp, 0.0)
            P_aw_insp = (flow_total + (self.paciente.E1 * V1 / self.paciente.R1) + (self.paciente.E2 * V2 / self.paciente.R2)) / ((1.0 / self.paciente.R1) + (1.0 / self.paciente.R2))
            P_aw = np.where(en_insp, P_aw_insp, self.ventilador.PEEP)
        elif self.ventilador.modo == 'PCV':
            # Lógica para PCV
            P_aw = self.ventilador.presion(t)
        else:
            raise ValueError(f"Modo desconocido: {self.ventilador.modo}")

        # Se calculan las derivadas dV/dt (sin cambios)
        dV1_dt = (P_aw - self.paciente.E1 * V1) / self.paciente.R1
        dV2_dt = (P_aw - self.paciente.E2 * V2) / self.paciente.R2

        return [dV1_dt, dV2_dt]

    def simular(self,
                num_ciclos: int=15,
                pasos_por_ciclo: int = 100):
        """Ejecuta múltiples ciclos y devuelve t, V1 y V2 concatenados."""
        t_data, V1_data, V2_data = [], [], []
        V0 = [0.0, 0.0]
        for i in range(num_ciclos):
            t0 = i * self.ventilador.T_total
            t1 = (i + 1) * self.ventilador.T_total
            if i < num_ciclos-1: #corrección para evitar discontinuidad en t=3.0
                t_eval = np.linspace(t0, t1, pasos_por_ciclo, endpoint=False)
            else:
                t_eval = np.linspace(t0, t1, pasos_por_ciclo, endpoint=True)
            sol = solve_ivp(self._modelo_edo,
                            [t0, t1],
                            V0,
                            t_eval=t_eval)
            t_data.append(sol.t)
            V1_data.append(sol.y[0])
            V2_data.append(sol.y[1])
            V0 = sol.y[:, -1]
        t = np.concatenate(t_data)
        V1 = np.concatenate(V1_data)
        V2 = np.concatenate(V2_data)
        return t, V1, V2

    def procesar_resultados(self,
                            t: np.ndarray,
                            V1: np.ndarray,
                            V2: np.ndarray) -> dict:
        """Calcula flujo, volumen total y presión resultante."""
        flujo1 = np.gradient(V1, t)
        flujo2 = np.gradient(V2, t)
        flujo_total = flujo1 + flujo2
        Vt = V1 + V2
        if self.ventilador.modo == 'PCV':
            P_aw = self.ventilador.presion(t)
        else:
            P_aw = ((flujo_total
                     + (self.paciente.E1 * V1 / self.paciente.R1)
                     + (self.paciente.E2 * V2 / self.paciente.R2))
                    / ((1 / self.paciente.R1) + (1 / self.paciente.R2)))
        return {
            't': t,
            'V1': V1,
            'V2': V2,
            'Vt': Vt,
            'flow1': flujo1,
            'flow2': flujo2,
            'flow': flujo_total,
            'P_aw': P_aw
        }

    def graficar_resultados(self,
                            resultados: dict,
                            titulo: str = 'Simulación Pulmonar'):
        """Grafica presión, flujo total y volumen total a partir del diccionario
        de resultados."""
        t = resultados['t']
        P_aw = resultados['P_aw']
        flujo = resultados['flow']
        Vt = resultados['Vt']

        fig, axs = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
        fig.suptitle(titulo, fontsize=16)

        # Presión
        axs[0].plot(t, P_aw, color='red', label='Presión (P_aw)')
        axs[0].set_ylabel('Presión (cmH2O)')
        axs[0].legend()

        # Flujo
        axs[1].plot(t, flujo, color='blue', label='Flujo Total')
        axs[1].set_ylabel('Flujo (L/s)')
        axs[1].axhline(0, color='grey', linewidth=0.8)
        axs[1].legend()

        # Volumen
        axs[2].plot(t, Vt, color='green', label='Volumen (L)')
        axs[2].set_ylabel('Volumen (L)')
        axs[2].set_xlabel('Tiempo (s)')
        axs[2].legend()

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

In [5]:
class IntercambioGases:
    """
    Módulo de intercambio gaseoso alveolar.
    Calcula presiones de CO2 y O2 alveolares a partir de los resultados
    de la simulación de mecánica respiratoria.
    """
    def __init__(
        self,
        ventilador,
        V_D: float,
        VCO2: float,
        R: float,
        FiO2: float,
        Pb: float,
        PH2O: float = 47.0,
        K: float = 0.863
    ):
        self.ventilador = ventilador
        self.V_D = V_D      # volumen muerto anatómico (L)
        self.VCO2 = VCO2    # producción de CO2 (L/min)
        self.R = R          # cociente respiratorio (adimensional)
        self.FiO2 = FiO2    # fracción inspirada de O2 (0-1)
        self.Pb = Pb        # presión barométrica (mmHg)
        self.PH2O = PH2O    # presión vapor de agua a 37°C (mmHg)
        self.K = K          # constante de conversión de unidades

    def calcular(self, resultados: dict) -> dict:
        """
        Ejecuta el cálculo de intercambio gaseoso.

        Parámetros
        ----------
        resultados : dict
            Diccionario de salida de Simulador.procesar_resultados(),
            con claves 't' (tiempo) y 'Vt' (volumen total alveolar, L).

        Devuelve
        -------
        dict con:
            VE_min: ventilación minuto total (L/min)
            VA_min: ventilación minuto alveolar (L/min)
            PACO2_mmHg: presión alveolar de CO2 (mmHg)
            PAO2_mmHg: presión alveolar de O2 (mmHg)
        """
        # 1. Frecuencia respiratoria (ciclos/min)
        f = self.ventilador.fr

        # 2. Volumen tidal (L)
        if self.ventilador.modo == 'VCV':
            VT = self.ventilador.Vt
        else:
            arr = resultados['Vt']
            VT = np.max(arr) - np.min(arr)

        # 3. Ventilaciones minuto
        VE = VT * f
        VA = (VT - self.V_D) * f
        if VA <= 0:
            raise ValueError("Ventilación alveolar ≤ 0, revisa V_D o simulación.")

        # 4. Presión alveolar de CO2 (mmHg)
        PACO2 = (self.VCO2 * self.K) / VA

        # 5. Presión alveolar de O2 (mmHg)
        PIO2 = self.FiO2 * (self.Pb - self.PH2O)
        PAO2 = PIO2 - (PACO2 / self.R)

        return {
            'VE_min': VE,
            'VA_min': VA,
            'PACO2_mmHg': PACO2,
            'PAO2_mmHg': PAO2
        }


In [6]:
class InteraccionCorazonPulmon:
    """
    Módulo de interacción hemodinámica corazón-pulmón.

    Modela el efecto de la presión en la vía aérea sobre el gasto cardíaco y la
    entrega de oxígeno.
    """
    def __init__(
        self,
        GC_base_L_min: float = 5.0,
        k_sensibilidad: float = 0.1,
        hb_g_dl: float = 15.0
    ):
        """
        Inicializa el estado cardiovascular basal del paciente.

        Parámetros
        ----------
        GC_base_L_min : float
            Gasto cardíaco basal del paciente en L/min.
        k_sensibilidad : float
            Factor de sensibilidad hemodinámica a la presión intratorácica.
            Un valor bajo (~0.05-0.1) simula un paciente normovolémico.
            Un valor alto (>0.2) simula un paciente hipovolémico o con disfunción cardíaca.
        hb_g_dl : float
            Concentración de hemoglobina en g/dL.
        """
        self.GC_base_L_min = GC_base_L_min
        self.k_sensibilidad = k_sensibilidad
        self.hb_g_dl = hb_g_dl
        # Constantes fisiológicas
        self.O2_CAP_HB = 1.34  # Capacidad de O2 por gramo de Hb (mL O2/g Hb)
        self.O2_SOL_PLASMA = 0.003  # Solubilidad de O2 en plasma (mL O2/dL/mmHg)

    def _estimar_sao2(self, pao2: float) -> float:
        """
        Estima la SaO2 a partir de la PaO2 usando una aproximación simple.
        NOTA: Simplificación educativa que no implementa la curva de disociación
        de la hemoglobina (Ecuación de Hill).
        """
        if pao2 >= 100:
            return 1.0
        elif pao2 >= 60:
            # Aproximación lineal груба entre 90% (a 60 mmHg) y 100% (a 100 mmHg)
            return 0.90 + 0.10 * ((pao2 - 60) / 40)
        else:
            # Aproximación para hipoxemia severa
            return 0.90 * (pao2 / 60)

    def calcular(
        self,
        resultados_mecanica: dict,
        resultados_gases: dict,
        ventilador: Ventilador
    ) -> dict:
        """
        Calcula el impacto hemodinámico de la ventilación mecánica.

        Parámetros
        ----------
        resultados_mecanica : dict
            El diccionario de salida de Simulador.procesar_resultados().
        resultados_gases : dict
            El diccionario de salida de IntercambioGases.calcular().
        ventilador : Ventilador
            La instancia del ventilador para obtener el PEEP.

        Devuelve
        -------
        dict con los resultados cardiovasculares:
            P_mean_cmH2O: Presión media en la vía aérea calculada.
            GC_actual_L_min: Gasto cardíaco resultante.
            PaO2_mmHg: Presión arterial de O2 estimada.
            SaO2_percent: Saturación arterial de O2 estimada.
            CAO2_ml_dl: Contenido arterial de O2.
            DO2_ml_min: Entrega de oxígeno a los tejidos.
        """
        t = resultados_mecanica['t']
        P_aw = resultados_mecanica['P_aw']
        PAO2_mmHg = resultados_gases['PAO2_mmHg']

        # 1. Calcular Presión Media en la Vía Aérea (P_mean)
        # Se integra el área bajo la curva de presión y se divide por la duración
        tiempo_total_ciclo = t[-1] - t[-2-1] if len(t)>1 else t[-1]
        p_aw_ultimo_ciclo = P_aw[t >= t[-1] - tiempo_total_ciclo]
        t_ultimo_ciclo = t[t >= t[-1] - tiempo_total_ciclo]

        area_bajo_curva = np.trapz(p_aw_ultimo_ciclo, t_ultimo_ciclo)
        P_mean = area_bajo_curva / (t_ultimo_ciclo[-1] - t_ultimo_ciclo[0])

        # 2. Calcular Gasto Cardíaco Actual
        # GC_actual = GC_base - k * (P_mean - PEEP_base)
        PEEP_base = ventilador.PEEP
        delta_p = P_mean - PEEP_base
        reduccion_gc = self.k_sensibilidad * delta_p
        GC_actual = self.GC_base_L_min - reduccion_gc
        # Asegurar que el GC no sea negativo
        GC_actual = max(0, GC_actual)

        # 3. Calcular Contenido Arterial de O2 (CAO2)
        # Se asume un gradiente Alveolo-arterial de O2 de 10 mmHg (simplificación)
        PaO2 = PAO2_mmHg - 10
        SaO2 = self._estimar_sao2(PaO2)

        # CAO2 = (Hb * SaO2 * 1.34) + (PaO2 * 0.003)
        O2_unido_hb = self.hb_g_dl * SaO2 * self.O2_CAP_HB
        O2_disuelto = PaO2 * self.O2_SOL_PLASMA
        CAO2_ml_dl = O2_unido_hb + O2_disuelto

        # 4. Entrega de Oxígeno (DO2)
        # DO2 (mL/min) = GC (L/min) * CAO2 (mL/dL) * 10 (dL/L)
        DO2_ml_min = GC_actual * CAO2_ml_dl * 10

        return {
            'P_mean_cmH2O': P_mean,
            'GC_actual_L_min': GC_actual,
            'PaO2_mmHg': PaO2,
            'SaO2_percent': SaO2 * 100,
            'CAO2_ml_dl': CAO2_ml_dl,
            'DO2_ml_min': DO2_ml_min
        }

Para la modelación del control del sistema respiratorio, el objetivo es modelar el ciclo de retroalimentación negativa fundamental: un aumento en la $P_a CO_2$ es detectado por los quimiorreceptores, lo que aumenta el impulso ventilatorio (el "drive to breathe") para incrementar la ventilación alveolar ($\dot{V}A$) y así normalizar la $P_a CO_2$. Los módulos que hemos definido previamente nos proporcionan todas las herramientas necesarias para simular este ciclo:
1. **Variable sensada**: El módulo de intercambio gaseoso calcula la presión alveolar de CO₂ ($P_A CO_2$), la cual es un sustituto bastante cercano de la $P_a CO_2$ para modelar la señal aferente del sistema de control.
1. **Señal de salida** (Impulso ventilatorio): El módulo mecánico puede simular la respiración espontánea cuando es accionado por una presión muscular ($P_{mus}t$). Por tanto, la salida del módulo de control será esta señal de presión muscular, que representa la respuesta eferente del sistema.

Se seleccionó entonces, un modelo de control basado en un controlador proporcional simple. Este enfoque, aunque es una simplificación de las complejas redes neuronales del centro respiratorio, se considera apropiada desde un punto de vista pedagógico. El modelo ajustará la amplitud y la frecuencia de la señal de presión muscular en proporción al "error" entre la PACO2 actual y un valor de referencia (${P_A CO_2}_{target}$). Las ecuaciones que gobernarán el módulo de control son:

$$\text{Amplitud de Pmus} = Gp \cdot (PACO2-PACO2target)$$

$$f_{resp}=fbase+Gf \cdot (PACO2-PACO2target)$$

Donde ${P_A CO_2}_{target}$ es el valor de referencia para la presión alveolar de CO₂ (40 mmHg), $f_{base}$ es la frecuencia respiratoria basal en reposo y $G_p$ y $G_f$ son las ganancias del controlador para la amplitud y la frecuencia, respectivamente.  

Estos parámetros permitirán al usuario simular diferentes sensibilidades del centro respiratorio. Este modelo se selecciona por su capacidad para crear un ciclo de retroalimentación funcional y educativamente claro con una complejidad computacional mínima. Permite al estudiante observar directamente cómo una alteración en la mecánica (que afecta la $\dot{V}A$) provoca un cambio en la $P_A CO_2$, el cual a su vez desencadena una respuesta compensatoria en el impulso ventilatorio, cerrando el ciclo homeostático.


In [7]:
class ControlRespiratorio:
    """
    Modelo de control respiratorio por retroalimentación negativa simple.

    Ajusta la amplitud y frecuencia de la presión muscular (P_mus)
    en función del error entre la PaCO2 actual (aproximada como PACO2)
    y un valor de referencia (PACO2_target).

    Ecuaciones:
        A = Gp * (PACO2 - PACO2_target)
        f = f_base + Gf * (PACO2 - PACO2_target)

    Métodos
    -------
    actualizar(PACO2)
        Calcula amplitud y frecuencia actuales del P_mus.
    generar_Pmus(t)
        Genera la señal P_mus(t) = A * sin(2π f t).
    """
    def __init__(self,
                 PACO2_target: float = 40.0,
                 f_base: float = 12.0,
                 Gp: float = 0.5,
                 Gf: float = 0.1):
        # Valor de referencia de PaCO2 (mmHg)
        self.PACO2_target = PACO2_target
        # Frecuencia respiratoria basal (ciclos/min)
        self.f_base = f_base
        # Ganancia para amplitud de P_mus (cmH2O por mmHg)
        self.Gp = Gp
        # Ganancia para frecuencia respiratoria (Hz por mmHg)
        # Convertir Gf de (ciclos/min)/mmHg a Hz/mmHg: Gf/60
        self.Gf = Gf / 60.0
        # Variables de estado
        self.amplitud = None
        self.frecuencia = None

    def actualizar(self, PACO2: float):
        """
        Actualiza la amplitud y frecuencia de P_mus en base a la PACO2.
        """
        error = PACO2 - self.PACO2_target
        # Amplitud en cmH2O
        self.amplitud = max(0.0, self.Gp * error)
        # Frecuencia en Hz
        self.frecuencia = max(0.1, self.f_base/60.0 + self.Gf * error)
        return self.amplitud, self.frecuencia

    def generar_Pmus(self, t: np.ndarray) -> np.ndarray:
        """
        Genera la señal de presión muscular sinusoidal:
        P_mus(t) = A * sin(2π f t)

        Parámetros
        ----------
        t : np.ndarray
            Vector de tiempos en segundos.

        Retorna
        -------
        np.ndarray
            Señal P_mus en cmH2O.
        """
        if self.amplitud is None or self.frecuencia is None:
            raise RuntimeError("Primero debe llamar a actualizar(PACO2)")
        omega = 2 * np.pi * self.frecuencia
        return self.amplitud * np.sin(omega * t)

Para poder implementar la ventilación espontánea, se requiere una modificación en la clase simulador, este modificación se hizo ya más arriba.

Y un ejemplo de implementación:

In [8]:
if __name__ == "__main__":

    # Paciente con una patología obstructiva leve (R1 y R2 aumentadas)
    paciente_obstructivo = Paciente(R1=15, C1=0.06, R2=20, C2=0.06)

    # Ventilador en modo espontáneo. Los parámetros de FR y Ti no se usarán
    # directamente, pero T_total define la duración de cada período de simulación.
    ventilador_esp = Ventilador(modo='ESP', PEEP=0)

    # Módulo de Control Respiratorio
    control = ControlRespiratorio(
        PACO2_target=40.0,
        f_base=12.0,       # Frecuencia basal de 12 rpm
        Gp=0.8,            # Ganancia de amplitud relativamente alta
        Gf=0.5             # Ganancia de frecuencia moderada
    )

    # Módulos de intercambio y cardiovascular
    intercambio = IntercambioGases(ventilador_esp, V_D=0.150, VCO2=0.2, R=0.8, FiO2=0.21, Pb=560.0)
    cardio = InteraccionCorazonPulmon(GC_base_L_min=5.0, k_sensibilidad=0.05, hb_g_dl=15.0)

    # Simulador que integra todos los componentes
    simulador_esp = Simulador(paciente_obstructivo, ventilador_esp, control)

    print(" Iniciando simulación de control respiratorio en lazo cerrado")

    num_iteraciones = 15
    duracion_chunk_s = 10.0 # Cada iteración simula 10 segundos de respiración

    # Estado inicial: el paciente está hipercápnico
    PACO2_actual = 55.0
    V0 = [0.0, 0.0] # Volúmenes iniciales en los compartimentos
    t_global = 0.0

    # Historial para graficar la evolución
    historial_tiempo = [0]
    historial_PACO2 = [PACO2_actual]
    historial_fr = []

    for i in range(num_iteraciones):
        # 1. El controlador define el patrón respiratorio basado en la PACO2
        amplitud, frecuencia_hz = control.actualizar(PACO2_actual)
        frecuencia_rpm = frecuencia_hz * 60
        historial_fr.append(frecuencia_rpm)

        print(f"\nIteración {i+1}/{num_iteraciones}:")
        print(f"  PACO2 inicial: {PACO2_actual:.1f} mmHg")
        print(f"  Control -> Amplitud: {amplitud:.1f} cmH2O, Frecuencia: {frecuencia_rpm:.1f} rpm")

        # 2. Simular un período de tiempo con ese patrón respiratorio
        t_start = t_global
        t_end = t_global + duracion_chunk_s
        t_eval = np.linspace(t_start, t_end, 500)

        # Usamos directamente el solver de EDO con el modelo del simulador
        sol = solve_ivp(
            simulador_esp._modelo_edo,
            [t_start, t_end],
            V0,
            t_eval=t_eval,
            dense_output=True
        )

        # 3. Procesar resultados para obtener la nueva PACO2
        resultados_mecanica = simulador_esp.procesar_resultados(sol.t, sol.y[0], sol.y[1])

        # Necesitamos la FR efectiva para el cálculo de IntercambioGases
        intercambio.ventilador.fr = frecuencia_rpm
        resultados_gases = intercambio.calcular(resultados_mecanica)

        # 4. Actualizar el estado para la siguiente iteración
        PACO2_actual = resultados_gases['PACO2_mmHg']
        V0 = sol.y[:, -1] # Usar el último estado de volumen como condición inicial
        t_global = t_end

        # Guardar en historial
        historial_tiempo.append(t_global)
        historial_PACO2.append(PACO2_actual)

    print("\n Simulación completada.")

    # --- VISUALIZACIÓN DE RESULTADOS ---
    fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    fig.suptitle('Evolución del Sistema de Control Respiratorio', fontsize=16)

    # Gráfico de PACO2
    axs[0].plot(historial_tiempo, historial_PACO2, 'o-', color='blue', label='PACO2 simulada')
    axs[0].axhline(y=control.PACO2_target, color='red', linestyle='--', label=f'PACO2 Objetivo ({control.PACO2_target} mmHg)')
    axs[0].set_ylabel('PACO2 (mmHg)')
    axs[0].legend()
    axs[0].set_title('Convergencia de la PACO2 hacia el valor de referencia')

    # Gráfico de Frecuencia Respiratoria
    tiempo_fr = np.array(historial_tiempo[:-1])
    axs[1].step(tiempo_fr, historial_fr, where='post', color='green', label='Frecuencia Respiratoria')
    axs[1].set_ylabel('Frecuencia (rpm)')
    axs[1].set_xlabel('Tiempo de simulación (s)')
    axs[1].legend()
    axs[1].set_title('Respuesta del Impulso Ventilatorio (Frecuencia)')

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

 Iniciando simulación de control respiratorio en lazo cerrado

Iteración 1/15:
  PACO2 inicial: 55.0 mmHg
  Control -> Amplitud: 12.0 cmH2O, Frecuencia: 19.5 rpm

Iteración 2/15:
  PACO2 inicial: 0.0 mmHg
  Control -> Amplitud: 0.0 cmH2O, Frecuencia: 6.0 rpm

Iteración 3/15:
  PACO2 inicial: 0.2 mmHg
  Control -> Amplitud: 0.0 cmH2O, Frecuencia: 6.0 rpm


ValueError: Ventilación alveolar ≤ 0, revisa V_D o simulación.